In [18]:
import sys, os
from typing import DefaultDict

In [19]:

def parse_bw(filepath, target):
    if target < 0 or target > 5:
        print('Wrong target range!')
        sys.exit(1)
        
    """
    target
    0: msg_size
    1: iter
    2: bw_peak
    3: bw_avg
    4: msg_rate
    5: all
    """
    res = []
    
    f = open(filepath, "r")
    lines = f.readlines()
    
    startline = 0
    for j in range(len(lines)):
        if lines[j][0:4] == " #by":
            startline = j + 1
            break
    endline = startline
    while endline < len(lines):
        if lines[endline][1:4] == '===':
            break
        endline += 1
        
    curline = startline
    while curline < endline:
        if lines[curline][0:2] == ' --':
            break
        if lines[curline][0] != ' ':
            curline += 1
            continue
        line = lines[curline].split()
        try:            
            # (msg_size, iteration, bw_peak, bw_avg, msg_rate)
            row = []
            if target == 5:
                row.append(line[0])
                row.append(line[1])
                row.append(line[2])
                row.append(line[3])
                row.append(line[4])
            else:
                row.append(line[target])
            res.append(row)
        except:
            curline += 1
            continue
        curline += 1
    
    return res

def parse_lat(filepath):
    f = open(filepath, "r")
    lines = f.readlines()
    
    startline = 0
    for j in range(len(lines)):
        if lines[j][0:4] == " #by":
            startline = j + 1
            break
    endline = startline
    while endline < len(lines):
        if lines[endline][1:4] == '===':
            break
        endline += 1
        
    curline = startline
    while curline < endline:
        if lines[curline][0:2] == ' --':
            break
        if lines[curline][0] != ' ':
            curline += 1
            continue
        line = lines[curline].split()
        try:            
            return line
                
        except:
            curline += 1
            continue

In [20]:

# Output format = [[ data1 ], [data2], ... , columns = [] ]
dataset = []
columns = ['rnic', 'test_node', 'test_type', 'tx_depth', 'mtu', 'bg_msgsize', 'bg_bw', 'msgsize_tput', 'iter_tput', 'bw_peak_tput', 'bw_avg_tput', 'msgrate_tput', 'msgsize_lat', 'iter_lat', 't_min', 't_max', 't_typical', 't_avg', 't_stdev', '99th_percentile', '99.9th_percentile']

target_dir = './test_result'
if os.path.isdir(target_dir) == False:
    print("Directory is not valid!")
    sys.exit()
    
file_list = os.listdir(target_dir)

### Get target files ###
target_list = []
bw_t = ['rb', 'wb', 'sb']
for target in file_list:
    splt_target = target.split('_')
    if splt_target[2] in bw_t and splt_target[6] == 'tf':
        target_list.append(target)

for t in target_list:
    print(t)
    row = []
    splt_t = t.split('_')
    
    # add testing env
    row.append(splt_t[0])
    row.append(splt_t[1])
    row.append(splt_t[2])
    row.append(splt_t[3][1:])
    row.append(splt_t[4][1:])
    row.append(splt_t[5][2:])
    
    # add bg_bw
    bws = parse_bw(target_dir + '/' + t[:-2] + 'bf', 3)
    row.append(bws[int(len(bws)/2)][0])
    
    # add tput data
    tput_data = parse_bw(target_dir + '/' + t, 5)[0]
    for e in tput_data:
        row.append(e)
    
    # add latency data
    try:
        lat_data = parse_lat(target_dir + '/' + t.split('b_')[0] + 'l_' + t.split('b_')[1])
    except:
        print('File Not Exist: ' + target_dir + '/' + t.split('b_')[0] + 'l_' + t.split('b_')[1])
        lat_data = [None] * 9
    
    for e in lat_data:
        row.append(e)

    print(row)
    dataset.append(row)
    # break

print(dataset)

X5_Y303_rb_t128_m1024_bg1024_tf
['X5', 'Y303', 'rb', '128', '1024', '1024', '3949.39', '16', '100000000', '0.00', '61.82', '4.051262', '16', '10000000', '2.02', '19.70', '3.60', '3.67', '0.43', '4.94', '9.04']
X5_Y303_rb_t128_m1024_bg1048576_tf
['X5', 'Y303', 'rb', '128', '1024', '1048576', '4139.78', '16', '100000000', '0.00', '37.21', '2.438696', '16', '10000000', '2.00', '23.40', '7.11', '7.11', '0.31', '8.59', '11.03']
X5_Y303_rb_t128_m1024_bg1073741824_tf
['X5', 'Y303', 'rb', '128', '1024', '1073741824', '4095.98', '16', '100000000', '0.00', '45.28', '2.967586', '16', '10000000', '1.92', '21.08', '2.40', '4.55', '2.57', '7.43', '10.80']
X5_Y303_rb_t128_m1024_bg16384_tf
['X5', 'Y303', 'rb', '128', '1024', '16384', '4137.03', '16', '100000000', '0.00', '37.36', '2.448684', '16', '10000000', '2.06', '22.46', '7.13', '7.17', '0.31', '8.83', '11.55']
X5_Y303_rb_t128_m1024_bg2048_tf
['X5', 'Y303', 'rb', '128', '1024', '2048', '4111.74', '16', '100000000', '0.00', '38.95', '2.552638', '1

In [21]:
import pandas as pd

df = pd.DataFrame(dataset, columns=columns)

print(df)
df.to_csv('X5.csv')

    rnic test_node test_type tx_depth   mtu  bg_msgsize    bg_bw msgsize_tput  \
0     X5      Y303        rb      128  1024        1024  3949.39           16   
1     X5      Y303        rb      128  1024     1048576  4139.78           16   
2     X5      Y303        rb      128  1024  1073741824  4095.98           16   
3     X5      Y303        rb      128  1024       16384  4137.03           16   
4     X5      Y303        rb      128  1024        2048  4111.74           16   
..   ...       ...       ...      ...   ...         ...      ...          ...   
355   X5      Y303        wb        2   512         256   173.23           16   
356   X5      Y303        wb        2   512       32768  3448.94           16   
357   X5      Y303        wb        2   512        4096  1913.37           16   
358   X5      Y303        wb        2   512         512   334.32           16   
359   X5      Y303        wb        2   512        8192  2626.75           16   

     iter_tput bw_peak_tput